# **redis-feast-gcp**: 03 - Triton + Vertex AI Prediction Inference Example

Here we will test the deployed Triton model on the Vertex AI Prediction endpoint.

**This notebook assumes that you've already set up your feature store, model repo in GCP, and deployed your model in Vertex AI with NVIDIA Triton**

![architecture](img/redis-feast-gcp-architecture.png)

## Create Inference Instances

Before we can test the Vertex AI Prediction endpoint, we need to construct a JSON body that represents an inference request. See the example below:

In [ ]:
import json

payload = {
    "id": "1",
    "inputs": [
        {
            "name": "INPUT",        ## Triton model input name
            "shape": [1, 1],        ## Triton model input shape
            "datatype": "BYTES",    ## Triton model input datatype
            "data": [["Virginia"]]  ## Triton model input data
        }
    ]
}

# Save to file
with open("instances.json", "w") as f:
    json.dump(payload, f)


## Test Endpoint

You can test the Vertex AI Prediction `rawPredict` endpoint using any HTTP tool or library, including `curl`.

In [ ]:
# Log in to GCloud using the CLI and your service account
!gcloud auth activate-service-account $SERVICE_ACCOUNT_EMAIL \
    --key-file=$GOOGLE_APPLICATION_CREDENTIALS \
    --project=$PROJECT_ID

In [ ]:
!echo $(gcloud ai endpoints list \
  --region=$GCP_REGION \
  --filter=display_name=vaccine-predictor-endpoint \
  --format="value(name)")

In [ ]:
%%bash

# Fetch Token
TOKEN=$(gcloud auth print-access-token)

# Fetch the Endpoint ID
ENDPOINT_ID=$(gcloud ai endpoints list \
  --region=$GCP_REGION \
  --filter=display_name=vaccine-predictor-endpoint \
  --format="value(name)")

# POST to the endpoint to get a response from the Triton ensemble model
curl \
  -X POST \
  -H "Authorization: Bearer ${TOKEN}" \
  -H "Content-Type: application/json" \
  https://${GCP_REGION}-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-east1/endpoints/${ENDPOINT_ID}:rawPredict \
  -d "@instances.json"
